### Doc2Vec Model

Gensim's Doc2Vec model vectorizes a group of words together rather than single words.
[Source](https://www.machinelearningplus.com/nlp/gensim-tutorial/#15howtoupdateanexistingword2vecmodelwithnewdata)

In [69]:
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from gensim import corpora
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from Bibor_PreProcessing import tokenizer_function

In [14]:
# code and idea adapted from Clay Carson, Jollene Muncy and Cynthia Chiang

# read in tokenized titles file
# tokenized_titles = pd.read_csv('./data/tokenized_titles.csv')

# read in tokenized combined file
# tokenized_combined = pd.read_csv('./data/tokenized_combined.csv')

In [58]:
# read in raw data
data = pd.read_csv('./data/combined.csv')

In [57]:
# set display options to full column widths
pd.set_option('display.max_colwidth', None)

In [71]:
# define function to tokenize a column

def tokenizer_function(column):
    """
    Takes in a text column
        tokenizes the text in each row
        using pattern [[a-zA-Z]\w+]
        which matches every lowercase and upperase character between a-z that are word characters
    Returns list of strings
    """
    
    # instantiate empty list of tokenized text
    texts = []
    
    # define tokenizer pattern
    pattern = '[a-zA-Z]\w+'
    # instantiate tokenizer
    tokenizer = RegexpTokenizer(pattern=pattern)
    
    # create for loop to tokenize each row and add the list of tokens to texts
    for text in column:
        tokens = tokenizer.tokenize(text)
        
        # transform tokens into lower case strings
        tokens = [token.lower() for token in tokens]
        texts.append(tokens)
    return texts

In [72]:
tokenized_titles = tokenizer_function(data['title'])

In [18]:
# convert pd.DataFrame into list
# tokenized_titles = tokenized_titles.values.tolist()

# source: https://note.nkmk.me/en/python-pandas-list/#:~:text=Convert%20labels%20(row%20%2F%20column%20names)%20to%20list,has%20a%20tolist()%20method.

In [73]:
type(tokenized_titles)
tokenized_titles[:2]

[['request',
  'for',
  'information',
  'new',
  'design',
  'booklet',
  'die',
  'cutting',
  'insert',
  'assembly'],
 ['audiovisual', 'suite', 'for', 'large', 'auditorium', 'maxwell', 'afb']]

**Create List of Tagged Documents**

[Source](https://www.machinelearningplus.com/nlp/gensim-tutorial/#15howtoupdateanexistingword2vecmodelwithnewdata)

In [74]:
# define function to create tagged documents
def tag_documents(list_of_documents):
    """generator function
       accepts list of tokenized documents
       in form of list of list of words
       attaches tags to each document
       yields list of tagged documents"""
    for i, document in enumerate(list_of_documents):
        yield TaggedDocument(document, [i])

In [75]:
# tag documents
tagged_titles = list(tag_documents(tokenized_titles))

In [76]:
tagged_titles[:1]

[TaggedDocument(words=['request', 'for', 'information', 'new', 'design', 'booklet', 'die', 'cutting', 'insert', 'assembly'], tags=[0])]

**Create Doc2Vec Model**

**--skip grams, vector size = 50, epochs = 20--**

In [77]:
# Instantiate the Doc2Vec model
# skip-gram model
model = Doc2Vec(vector_size=50, dm=0, dbow_words=1, min_count=2, seed=1977, epochs=20)

In [78]:
# Build the Volabulary
model.build_vocab(tagged_titles)

In [79]:
%%time
# Train the Doc2Vec model
model.train(tagged_titles, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1min 59s, sys: 16.5 s, total: 2min 15s
Wall time: 1min 51s


In [80]:
# Save model for future use
model.save('doc2vec_titles')

In [17]:
# Load saved doc2vec_model
# model = Doc2Vec.load('doc2vec_model')

**Get Similar Documents**

In [81]:
# get the 10 most similar documents to the first document
model.docvecs.most_similar(0, topn=10)

[(3092, 0.8073619604110718),
 (45192, 0.8061673641204834),
 (3337, 0.8005468249320984),
 (41958, 0.8004189729690552),
 (35197, 0.7944138646125793),
 (2295, 0.7916344404220581),
 (33808, 0.7906997799873352),
 (301, 0.7852705121040344),
 (6362, 0.7842591404914856),
 (16015, 0.7828555107116699)]

**Let's investigate how similar these documents are**

In [2]:
# read in dataframe of contract notifications
data = pd.read_csv('./data/combined.csv')
data.sample(25)

,noticeId,title,solicitationNumber,department,subTier,office,postedDate,type,baseType,archiveType,...,award,pointOfContact,description,organizationType,officeAddress,placeOfPerformance,additionalInfoLink,uiLink,links,resourceLinks
40819,dc5becef2cc64c46bd8306b130e084e6,56--VALVE BODY INSUL,N0010420QEF63,DEPT OF DEFENSE,DEPT OF THE NAVY,NAVSUP WEAPON SYSTEMS SUPPORT MECH,2020-05-28,Solicitation,Solicitation,auto15,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'LY...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '17050-0788', 'city': 'MECHANICSBU...",{},NaN,https://beta.sam.gov/opp/dc5becef2cc64c46bd830...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",NaN
21955,496f02eda0f94bd9a8b3e84b8a77815d,"47--ADAPTER,STRAIGHT,TUBE",SPE7M320U0899,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA LAND AND MARITIME,2020-05-07,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Di...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '43218-3990', 'city': 'COLUMBUS', ...",{},NaN,https://beta.sam.gov/opp/496f02eda0f94bd9a8b3e...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",NaN
2530,61c2bdb4aee540a3bdf42a3886aca240,Kanopolis Lake Vegetation Management,W912DQ20R1058,DEPT OF DEFENSE,DEPT OF THE ARMY,W071 ENDIST KANSAS CITY,2020-04-22,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,NaN,"[{'fax': '', 'type': 'primary', 'email': 'laur...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '64106-2896', 'city': 'KANSAS CITY...",NaN,NaN,https://beta.sam.gov/opp/61c2bdb4aee540a3bdf42...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",['https://beta.sam.gov/api/prod/opps/v3/opport...
12098,e39501dbb64a451fb5b10c8f933ccd6e,FD2020-20-00640,NaN,DEPT OF DEFENSE,DEPT OF THE AIR FORCE,FA8221 AFNWC PZBB,2020-03-11,Special Notice,Special Notice,autocustom,...,NaN,NaN,https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '84056-5837', 'city': 'HILL AFB', ...",NaN,NaN,https://beta.sam.gov/opp/e39501dbb64a451fb5b10...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",['https://beta.sam.gov/api/prod/opps/v3/opport...
38514,6ed1f768283b497bb19066368ecf53cd,41--POSI-VAC STARTER KI,SPE8E820T3561,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA TROOP SUPPORT,2020-05-18,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Di...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '19111-5096', 'city': 'PHILADELPHI...",{},NaN,https://beta.sam.gov/opp/6ed1f768283b497bb1906...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",NaN
40638,f8a0f16a5896412581cd81f84c552d3e,"43--PUMP UNIT,CENTRIFUG",SPE7M120T438A,DEPT OF DEFENSE,DEFENSE LOGISTICS AGENCY (DLA),DLA LAND AND MARITIME,2020-05-28,Combined Synopsis/Solicitation,Combined Synopsis/Solicitation,autocustom,...,{'awardee': {'location': {}}},"[{'fax': None, 'type': 'primary', 'email': 'Di...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '43218-3990', 'city': 'COLUMBUS', ...",{},NaN,https://beta.sam.gov/opp/f8a0f16a5896412581cd8...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",NaN
41257,99d96fbf1dee437b8acc864286598ba9,Delivery Order for Class I Engineering Change ...,N00019-20-RFPREQ-PMA-201-0259,DEPT OF DEFENSE,DEPT OF THE NAVY,NAVAL AIR SYSTEMS COMMAND,2020-05-28,Presolicitation,Presolicitation,autocustom,...,NaN,"[{'fax': '', 'type': 'primary', 'email': 'sama...",https://api.sam.gov/prod/opportunities/v1/noti...,OFFICE,"{'zipcode': '20670-5000', 'city': 'PATUXENT RI...","{'city': {'code': '65000', 'name': 'Saint Loui...",NaN,https://beta.sam.gov/opp/99d96fbf1dee437b8acc8...,"[{'rel': 'self', 'href': 'https://api.sam.gov/...",NaN
13097,ba00b2b5611d4153b5727a08cae93211,Serena Software - Sole Source (Including Brand...,

In [90]:
# find most similar documents in dataframe
def most_similar(path_to_model, base_document_idx, n):
    """function
       finds most similar n documents
       to base_document
       based on cosine similarity
       returns similar documents 
       as pandas dataframe"""
    # load saved doc2vec_model
    model = Doc2Vec.load(path_to_model)
    # get similar topics
    similars = model.docvecs.most_similar(base_document_idx, topn=n)
    
    # load original dataframe
    data = pd.read_csv('./data/combined.csv')
    # base document
    row1 = data.loc[base_document_idx, ['title', 'department', 'uiLink']]
    
    # list of rows in the original dataframe
    # initialized with the base_document
    list_of_dfs = [row1]
    
    # iterate through all similar notifications
    for tag, similar_document in similars:
        # find the row in the notifications dataframe corresponding to the tag
        df = data.loc[tag, ['title', 'department', 'uiLink']]
        # add row to the list of rows 
        list_of_dfs.append(df)
    # return all rows as a dataframe   
    return pd.DataFrame(list_of_dfs)

In [91]:
# show most similar 10 titles to the notification indexed 2530
path = 'doc2vec_titles'
most_similar(path, 2530, 10)

,title,department,uiLink
2530,Kanopolis Lake Vegetation Management,DEPT OF DEFENSE,https://beta.sam.gov/opp/61c2bdb4aee540a3bdf42a3886aca240/view
25496,Vegetation Services,DEPT OF DEFENSE,https://beta.sam.gov/opp/da7ea2a5c4f344ef898e254bff65be8e/view
23154,"HERBICIDE APPLICATION SERVICES AT CLINTON LAKE, IN LAWRENCE, KS",DEPT OF DEFENSE,https://beta.sam.gov/opp/2672fdbf215643599592dc9154239a28/view
30384,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/afb469ce250d4ef2bcacf858b89832c0/view
3673,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/48d632c59caa4e8997f61d6d5ab71c69/view
13667,"Security Gates, Lake Mead NRA","INTERIOR, DEPARTMENT OF THE",https://beta.sam.gov/opp/38799521cff148d487c6ba68b59a55b7/view
42693,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/afb469ce250d4ef2bcacf858b89832c0/view
46080,R.D. Bailey Lake Herbicide Spraying,DEPT OF DEFENSE,https://beta.sam.gov/opp/14eb1296719e44f587c470489ac3159f/view
34781,Benbrook Lake Mowing Services,DEPT OF DEFENSE,https://beta.sam.gov/opp/61ba07930a6f4a1986325a9bf3f3948e/view
20699,"W912BV20Q0056 - Herbicide Application Services for Project Lands at Tenkiller Lake, OK",DEPT OF DEFENSE,https://beta.sam.gov/opp/a2c5be6011a44a09a2624bc33e5d241f/view


**Create Doc2Vec Model**

**--cbow, vector size = 100, epochs = 20--**

In [92]:
# Instantiate the Doc2Vec model
# cbow model
model_cbow = Doc2Vec(vector_size=100, dm=1, dbow_words=0, seed=1977, epochs=20)

In [93]:
# Build the Volabulary
model_cbow.build_vocab(tagged_titles)

In [94]:
%%time
# Train the Doc2Vec model
model_cbow.train(tagged_titles, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1min 8s, sys: 9.04 s, total: 1min 17s
Wall time: 1min 7s


In [95]:
# Save model for future use
model.save('doc2vec_cbow_titles')

In [26]:
# Load saved doc2vec_model
# model = Doc2Vec.load('doc2vec_model')

**Get Similar Documents**

In [98]:
# get the 10 most similar documents to the first document
model_cbow.docvecs.most_similar(0, topn=10)

[(13895, 0.7264459729194641),
 (39655, 0.6743890047073364),
 (21008, 0.6409734487533569),
 (20032, 0.6245008111000061),
 (43718, 0.6221784949302673),
 (27529, 0.6027742624282837),
 (10617, 0.5990416407585144),
 (8238, 0.5871819257736206),
 (33513, 0.5820512771606445),
 (23279, 0.5790287256240845)]

In [99]:
path_cbow = 'doc2vec_cbow_titles'
most_similar(path_cbow, 2530, 10)

,title,department,uiLink
2530,Kanopolis Lake Vegetation Management,DEPT OF DEFENSE,https://beta.sam.gov/opp/61c2bdb4aee540a3bdf42a3886aca240/view
25496,Vegetation Services,DEPT OF DEFENSE,https://beta.sam.gov/opp/da7ea2a5c4f344ef898e254bff65be8e/view
23154,"HERBICIDE APPLICATION SERVICES AT CLINTON LAKE, IN LAWRENCE, KS",DEPT OF DEFENSE,https://beta.sam.gov/opp/2672fdbf215643599592dc9154239a28/view
30384,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/afb469ce250d4ef2bcacf858b89832c0/view
3673,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/48d632c59caa4e8997f61d6d5ab71c69/view
13667,"Security Gates, Lake Mead NRA","INTERIOR, DEPARTMENT OF THE",https://beta.sam.gov/opp/38799521cff148d487c6ba68b59a55b7/view
42693,Forestry Vegetation Management Treatment,"AGRICULTURE, DEPARTMENT OF",https://beta.sam.gov/opp/afb469ce250d4ef2bcacf858b89832c0/view
46080,R.D. Bailey Lake Herbicide Spraying,DEPT OF DEFENSE,https://beta.sam.gov/opp/14eb1296719e44f587c470489ac3159f/view
34781,Benbrook Lake Mowing Services,DEPT OF DEFENSE,https://beta.sam.gov/opp/61ba07930a6f4a1986325a9bf3f3948e/view
20699,"W912BV20Q0056 - Herbicide Application Services for Project Lands at Tenkiller Lake, OK",DEPT OF DEFENSE,https://beta.sam.gov/opp/a2c5be6011a44a09a2624bc33e5d241f/view


**Get Similar Words**

In [100]:
# inspect the vocabulary

vocabulary = model.wv.vocab
# vocabulary

In [106]:
# get the first 10 items in the vocabulary
list(vocabulary.keys())[:10]

['request',
 'for',
 'information',
 'new',
 'design',
 'booklet',
 'die',
 'cutting',
 'insert',
 'assembly']

In [107]:
# get the frequency of a word in the corpus 
model.wv.vocab['request'].count 

328

In [111]:
# get words similar to the entry term with their cosine similarity scores
model.wv.most_similar_cosmul('request')

[('information', 0.9256718158721924),
 ('proposals', 0.8846799731254578),
 ('agv', 0.8732286691665649),
 ('ppb2', 0.8524883389472961),
 ('r420', 0.849981427192688),
 ('adhesives', 0.8489277958869934),
 ('cornerstone', 0.8470085859298706),
 ('acclimation', 0.8461048007011414),
 ('papers', 0.8448440432548523),
 ('infill', 0.8435037732124329)]

The list of most similar words to request in the document is promising, but it could be better. More text cleaning is needed.

In [ ]:
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py

# when building a recommender - I can use the infer vector function - that takes a list of tokens returns the vector and that can be 
# compared to other vectors via cosine similarity
# vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])